In [68]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [67]:
url = 'https://en.wikipedia.org/wiki/Demographics_of_India'
response = requests.get(url)
response

<Response [200]>

In [78]:
# create soup
soup = BeautifulSoup(response.content, 'lxml')

In [80]:
# required table is under tag table and class = wikitable sortable
pop_dist = soup.find_all('table', class_ = 'wikitable sortable')[4]

In [81]:
states = []
# get states names from the table
for tag in pop_dist.find_all('td'):
    if tag.find('a'):
        states.append(tag.find('a').text)

In [82]:
print(states)

['Uttar Pradesh', 'Maharashtra', 'Bihar', 'West Bengal', 'Madhya Pradesh', 'Tamil Nadu', 'Rajasthan', 'Karnataka', 'Gujarat', 'Andhra Pradesh', 'Odisha', 'Telangana', 'Kerala', 'Jharkhand', 'Assam', 'Punjab', 'Chhattisgarh', 'Haryana', 'Delhi', 'Jammu and Kashmir', 'Uttarakhand', 'Himachal Pradesh', 'Tripura', 'Meghalaya', 'Manipur', 'Nagaland', 'Goa', 'Arunachal Pradesh', 'Puducherry', 'Mizoram', 'Chandigarh', 'Sikkim', 'Andaman and Nicobar Islands', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Lakshadweep']


Colleting data for the rest of the table.

Respective column data is available at every **12th step.**

Below is the Population Column data, eg.uttar pradesh has population of 199,812,341 etc


In [83]:
all_data = []
for i in range(2,12):
    all_data.append(pop_dist.find_all('td')[i::12])

In [84]:
# Create a function to get individual column data
def get_data(all_data):
    df_list = []
    for data in all_data:
        df_list.append(int(data.text.strip().replace(',','')))
    return df_list

In [85]:
df_dict = {0 : [], 2 : [], 3 : [], 4 : [], 5 : [], 6 : [], 7 : [], 8 : [], 9 : []}

In [86]:
# call the get_data function and store the column data in respective value of df_dict
for i in range(len(df_dict)+1):
    if i == 1:
        continue
    else:
        data = get_data(all_data[i])
        df_dict[i] = data

In [87]:
percent = []
for tag in all_data[1]:
    percent.append(float(tag.text.strip()))

In [88]:
states = pd.DataFrame(states,columns = ['states'])
percent = pd.DataFrame(percent,columns = ['percent'])
df = pd.DataFrame(df_dict)
df = pd.concat([states,df,percent],axis = 1)
df.tail()

,states,0,2,3,4,5,6,7,8,9,percent
31,Sikkim,610577,323070,287507,35563,890,455962,151726,7096,86,0.05
32,Andaman and Nicobar Islands,380581,202871,177710,25161,876,244411,135533,8249,46,0.03
33,Dadra and Nagar Haveli,343709,193760,149949,43811,774,183024,159829,491,698,0.03
34,Daman and Diu,243247,150301,92946,57355,618,60331,182580,112,2169,0.02
35,Lakshadweep,64473,33123,31350,1773,946,14121,50308,32,2013,0.01


In [90]:
df.columns = ['states','population','male','female','diff_bet_male_and_female','sex ratio','rural','urban','area','density','percent']
print("Number of observations in df {df.shape}")
df.head()

Number of observations in df {df.shape}


,states,population,male,female,diff_bet_male_and_female,sex ratio,rural,urban,area,density,percent
0,Uttar Pradesh,199812341,104480510,95331831,9148679,930,155111022,44470455,240928,828,16.50
1,Maharashtra,112374333,58243056,54131277,4111779,929,61545441,50827531,307713,365,9.28
2,Bihar,104099452,54278157,49821295,4456862,918,92075028,11729609,94163,1102,8.60
3,West Bengal,91276115,46809027,44467088,2341939,950,62213676,29134060,88752,1030,7.54
4,Madhya Pradesh,72626809,37612306,35014503,2597803,931,52537899,20059666,308245,236,6.00
